# 5.10 批量归一化

批量归一化（batch normalization）层，它能让较深的神经网络的训练变得更加容易 [1]。在实战Kaggle比赛：预测房价里，对输入数据做了标准化处理：处理后的任意一个特征在数据集中所有样本上的均值为0、标准差为1。标准化处理输入数据使各个特征的分布相近：这往往更容易训练出有效的模型。

通常来说，数据标准化预处理对于浅层模型就足够有效了。随着模型训练的进行，当每层中参数更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经网络来说，即使输入数据已做标准化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归一化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。批量归一化和下一节将要介绍的残差网络为训练和设计深度模型提供了两类重要思路。

批量归一化层和丢弃层一样，在训练模式和预测模式的计算结果是不一样的。

## 5.10.1 批量归一化层

### 5.10.1.1 对全连接层做批量归一化

通常，我们将批量归一化层置于全连接层中的仿射变换和激活函数之间。设全连接层的输入为$u$，权重参数和偏差参数分别为$W$和$b$，激活函数为$\phi $。设批量归一化的运算符为$BN$。那么，使用批量归一化的全连接层的输出为:$$\phi (BN(x)),$$

其中批量归一化$x$由放射变换$$x=Wu+b$$得到。

考虑一个由$m$个样本组成的小批量，仿射变换的输出为一个新的小批量$B={x^{(1)},...,x^{(m)}}$。它们正是批量归一化层的输入。对于小批量$B$中任意样本$x^(i)\in \mathbb R^{d},1\leqslant i \leqslant m,$批量归一化层的输出同样是$d$维向量:$$y^{(i)}=BN(x^{(i)}),$$

首先，对小批量$B$求均值和方差：
$$\mu _{B}\leftarrow \frac{1}{m}\sum_{i=1}^{m}x^{(i)},$$

$$\sigma _{B}^2\leftarrow \frac{1}{m}(x^{(i)}-\mu_B)^2$$


按元素开方和按元素除法对$x^{(i)}$标准化：

$$\hat{x^{(i)}}\leftarrow \frac{x^{(i)}-\mu_B}{\sqrt{\sigma_B^2+\epsilon }}$$

这里ϵ>0是一个很小的常数，保证分母大于0。在上面标准化的基础上，批量归一化层引入了两个可以学习的模型参数，拉伸（scale）参数 γ 和偏移（shift）参数 β。这两个参数和$x^{(i)}$形状相同，皆为$d$维向量。它们与$x^{(i)}$分别做按元素乘法（符号⊙）和加法计算：$$y^{(i)}\leftarrow \gamma \bigodot \hat{x^{(i)}}+\beta $$

我们得到了$x^(i)$的批量归一化的输出$y^(i)$。 值得注意的是，可学习的拉伸和偏移参数保留了不对$xˆ(i)$做批量归一化的可能：此时只需学出$\gamma =\sqrt{\sigma_B^2+\epsilon}$和$\beta =\mu_B$。$\gamma $用来调整方差，$\mu$用来调整均值。我们可以对此这样理解：如果批量归一化无益，理论上，学出的模型可以不使用批量归一化。

### 5.10.1.2 对卷积层做批量归一化

对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前。如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且每个通道都拥有独立的拉伸和偏移参数，并均为标量。设小批量中有$m$个样本。在单个通道上，假设卷积计算输出的高和宽分别为$p$和$q$。我们需要对该通道中$m×p×q$个元素同时做批量归一化。对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中$m×p×q$个元素的均值和方差。

### 5.10.1.3 预测时的批量归一化

使用批量归一化训练时，我们可以将批量大小设得大一点，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，我们希望模型对于任意输入都有确定的输出。因此，单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

## 5.10.2 从零开始实现

In [19]:
import time
import torch
from torch import nn,optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device=torch.device('cuda'if torch.cuda.is_available()else 'cpu')

def batch_norm(is_training,X,gamma,beta,moving_mean,moving_var,eps,momentum):
    #判断当前模式是训练模式还是预测模式
    if not is_training:
        #如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat=(X-moving_mean)/torch.sqrt(moving_var+eps)
    else:
        assert len(X.shape)in (2,4)
        if len(X.shape)==2:
            #使用全连接层的情况，计算特征维上的均值和方差
            mean=X.mean(dim=0)
            var=((X-mean)**2).mean(dim=0)
        else:
            #使用二维卷积层的情况，计算通道维(axis=1)的均值和方差
            #X的形状以便后面做广播运算
            #对单个通道的多个样本求均值，分别是样本维，行维和列维
            mean=X.mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
            var=((X-mean)**2).mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
        #训练模式下用当前的均值和方差做标准化
        X_hat=(X-mean)/torch.sqrt(var+eps)
        #更新移动平均的均值和方差
        moving_mean=momentum*moving_mean+(1.0-momentum)*mean
        moving_var=momentum*moving_var+(1.0-momentum)*var
    Y=gamma*X_hat+beta #拉伸和偏移
    return Y,moving_mean,moving_var

In [20]:
X=torch.ones(5,3,3)
print(X)
mean=X.mean(dim=0,keepdim=True).mean(dim=2,keepdim=True)
print(mean)

tensor([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]])
tensor([[[1.],
         [1.],
         [1.]]])


我们自定义一个BatchNorm层。它保存参与求梯度和迭代的拉伸参数gamma和偏移参数beta，同时也维护移动平均得到的均值和方差，以便能够在模型预测时被使用。BatchNorm实例所需指定的num_features参数对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数。该实例所需指定的num_dims参数对于全连接层和卷积层来说分别为2和4。

In [21]:
#对卷积层进行归一化

class BatchNorm(nn.Module):
    def __init__(self,num_features,num_dims):
        super(BatchNorm,self).__init__()
        if num_dims==2:
            shape=(1,num_features)
        else:
            shape=(1,num_features,1,1)
        #参与求梯度和迭代的拉伸和偏移参数，分别初始化为0和1
        self.gamma=nn.Parameter(torch.ones(shape))
        self.beta=nn.Parameter(torch.zeros(shape))
        #不参与求梯度和迭代的变量，全在内存上初始化为0
        self.moving_mean=torch.zeros(shape)
        self.moving_var=torch.zeros(shape)
        
    def forward(self,X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device!=X.device:
            self.moving_mean=self.moving_mean.to(X.device)
            self.moving_var=self.moving_var.to(X.device)
# 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y,self.moving_mean,self.moving_var=batch_norm(self.training,X,
                                self.gamma,self.beta,self.moving_mean,
                                self.moving_var,eps=1e-5,momentum=0.9)
        return Y

### 5.10.2.1 使用批量归一化层的LeNet

修改5.5节（卷积神经网络（LeNet））介绍的LeNet模型，从而应用批量归一化层。我们在所有的卷积层或全连接层之后、激活层之前加入批量归一化层。

In [22]:
net=nn.Sequential(
    nn.Conv2d(1,6,5),#in_channels,out_channels,kernel_size
    BatchNorm(6,num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2,2),#kernel_size,stride
    nn.Conv2d(6,16,5),
    BatchNorm(16,num_dims=4),#num_features,num_dims
    nn.Sigmoid(),
    nn.MaxPool2d(2,2),
    d2l.FlattenLayer(),
    nn.Linear(16*4*4,120),
    BatchNorm(120,num_dims=2),
    nn.Sigmoid(),
    nn.Linear(120,84),
    BatchNorm(84,num_dims=2),
    nn.Sigmoid(),
    nn.Linear(84,10)
)

In [ ]:
#训练修改后的模型
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size)

lr,num_epochs=0.001,5
optimizer=torch.optim.Adam(net.parameters(),lr=lr)
d2l.train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

In [ ]:
#查看第一个批量归一化层学习到的拉伸参数gamma和偏移参数beta

net[1].gamma.view((-1,)),net[1].beta.view((-1,))

### 5.10.3 简洁实现

Pytorch中nn模块定义的BatchNorm1d和BatchNorm2d类使用起来更加简单，二者分别用于全连接层和卷积层，都需要指定输入的num_features参数值。

In [23]:
net=nn.Sequential(
    nn.Conv2d(1,6,5),#in_channels,out_channels,kernel_size
    nn.BatchNorm2d(6),
    nn.Sigmoid(),
    nn.MaxPool2d(2,2),#kernel_size,stride
    nn.Conv2d(6,16,5),
    nn.BatchNorm2d(16),
    nn.Sigmoid(),
    nn.MaxPool2d(2,2),
    d2l.FlattenLayer(),
    nn.Linear(16*4*4,120),
    nn.BatchNorm1d(120),
    nn.Sigmoid(),
    nn.Linear(120,84),
    nn.BatchNorm1d(84),
    nn.Sigmoid(),
    nn.Linear(84,10)
)

In [ ]:
#使用同样的超参数进行训练
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size,root='../Datasets/FashionMNIST')

lr,num_epochs=0.001,5
optimizer=torch.optim.Adam(net.parameters(),lr=lr)
d2l.train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

参考文献

[1] Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training by reducing internal covariate shift. arXiv preprint arXiv:1502.03167.